# EBC-Var6: temperature-dependent parameters
In this notebook, I would use Arrhenius equation in every parameters (except efficiency terms) in temperature-dependent format.  The backbone model is shown as below:
![](graph/ebc6.png)
Arrhenius equation general format is:
\begin{equation*}
    k = A\cdot e^{-E_A/(RT_K)}
\end{equation*}
which $k$ is theoretical rate, $A$ is a pre-exponential factor, $E_A$ is the activation energy, $R$ is the universal gas constant and $T_K$ is the absolute temperature  

In this model, $k$ is replacing every rate parameters with $E_A$ is a species-specific energy parameter and $A$ are species- and parameter-specific normalization constant.
## lang & pkg
python 3.7.3  
sympy: 1.5.1, mpmath 1.5.0

In [1]:
from sympy import *
import math as mt
init_printing()

## vars & Eqs

In [2]:
C, P, B = symbols("C P B")
R = 8.31
T_k, E_P,e_P,g_0,r_0,m_0,a_0, E_B,e_B,g_8,r_8,m_8 = symbols("T_k E_P e_0 g_0 r_0 m_0 a_0 E_B e_B g_8 r_8 m_8", positive=True)

## parameters equations

In [3]:
g_P = g_0*exp(-E_P/(R*T_k))
r_P = r_0*exp(-E_P/(R*T_k))
m_P = m_0*exp(-E_P/(R*T_k))
a_P = a_0*exp(-E_P/(R*T_k))

g_B = g_8*exp(-E_B/(R*T_k))
r_B = r_8*exp(-E_B/(R*T_k))
m_B = m_8*exp(-E_B/(R*T_k))

# C_eqm = Eq((g_P*(1 -e_P) +a_P*P +m_P)*P + (g_B*(1 -e_B)*C +m_B)*B, 0)
# P_eqm = Eq((g_P*e_P -a_P*P -r_P -m_P)*P, 0)
# B_eqm = Eq((g_B*e_B*C -r_B -m_B)*B, 0)
g_P

    -0.120336943441637⋅E_P 
    ───────────────────────
              T_k          
g₀⋅ℯ                       

## equation system
\begin{equation}
\left\{\begin{array}{rl}
C'(t) &= (g_P(1-\varepsilon_P) + a_PP + m_P)P + (g_B(1-\varepsilon_B)C + m_B)B\\
P'(t) &= (g_P\varepsilon_P - a_PP - R_P - m_P)P\\
B'(t) &= (g_B\varepsilon_BC - R_B - m_B)B\\
\end{array}\right.
\end{equation}

## parameters table
symbol | unit | meaning
--- | --- | ---
$C$ | kgCm$^-3$ | system organic __C__arbon pool concentration
$P$ | kgCm$^-3$ | system __P__hytoplankton Carbon pool concentration
$B$ | kgCm$^-3$ | system _E. coli_ (__B__acteria) Carbon pool concentration
$a_P$ | $m^3(kgC)^{-1}$ | intraspecific density dependence coefficient
$g_P$ | t$^{-1}$ | growth rate of $P$
$e_P$ | no | carbon assimilation efficiency of $P$
$R_P$ | t$^{-1}$ | respiration rate of $P$
$m_P$ | t$^{-1}$ | death rate of $P$
$g_B$ | m$^3$(kgC$\cdot$t)$^{-1}$ | growth rate per $B$ pool concentration unit
$e_B$ | no | absorption efficiency of $B$
$R_B$ | t$^{-1}$ | respiration rate of $B$
$m_B$ | t$^{-1}$ | death rate of $B$

## equations simplification

In [4]:
C_eqm = (g_P*(1 -e_P) +a_P*P +m_P)*P + (g_B*(1 -e_B)*C +m_B)*B
P_eqm = (g_P*e_P -a_P*P -r_P -m_P)*P
B_eqm = (g_B*e_B*C -r_B -m_B)*B

C_sim = simplify(C_eqm)
P_sim = simplify(P_eqm)
B_sim = simplify(B_eqm)

C_sim,P_sim,B_sim

⎛⎛                           0.120336943441637⋅E_P                            
⎜⎜                           ─────────────────────                            
⎜⎜                                    T_k                                     
⎝⎝- B⋅(C⋅g₈⋅(e_B - 1) - m₈)⋅ℯ                      + P⋅(P⋅a₀ - g₀⋅(e₀ - 1) + m

    0.120336943441637⋅E_B⎞  -0.120336943441637⋅(E_B + E_P)                    
    ─────────────────────⎟  ───────────────────────────────                   
             T_k         ⎟                T_k                                 
₀)⋅ℯ                     ⎠⋅ℯ                               , -P⋅(P⋅a₀ - e₀⋅g₀ 

            -0.120336943441637⋅E_P                           -0.12033694344163
            ───────────────────────                          ─────────────────
                      T_k                                              T_k    
+ m₀ + r₀)⋅ℯ                       , B⋅(C⋅e_B⋅g₈ - m₈ - r₈)⋅ℯ                 

7⋅E_B ⎞
──────⎟
      ⎟
      ⎠

# simplified ODE equation system
\begin{equation}\left\{\begin{array}{rl}
    C'(t) &= e^{-\dfrac{E_B+E_P}{RT_k}}(P(P\cdot a_P-g_P(e_P-1)+m_P)e^{\dfrac{E_B}{RT_k}}-B(C\cdot g_B)e^{\dfrac{E_P}{RT_k}})\\
    P'(t) &= -P(P\cdot a_P-e_P\cdot g_P + m_P + r_P)e^{-\dfrac{E_P}{RT_k}}\\
    B'(t) &= B(C\cdot e_B\cdot g_B-m_B-r_B)e^{-\dfrac{E_B}{RT_k}}
\end{array}\right.\end{equation}

In [5]:
C_eqm = Eq(C_eqm,0)
P_eqm = Eq(P_eqm,0)
B_eqm = Eq(B_eqm,0)

sol = solve([C_eqm, P_eqm, B_eqm], [C, P, B])
sol

⎡               ⎛   e₀⋅g₀ - m₀ - r₀     ⎞  ⎛m₈ + r₈        ⎞  ⎛m₈ + r₈  e₀⋅g₀ 
⎢(C, 0.0, 0.0), ⎜C, ───────────────, 0.0⎟, ⎜───────, 0.0, B⎟, ⎜───────, ──────
⎣               ⎝          a₀           ⎠  ⎝ e_B⋅g₈        ⎠  ⎝ e_B⋅g₈        

- m₀ - r₀   ⎞⎤
─────────, B⎟⎥
 a₀         ⎠⎦

## extract coexistence system

In [6]:
sol[3] ## the only solution with coexistence

⎛m₈ + r₈  e₀⋅g₀ - m₀ - r₀   ⎞
⎜───────, ───────────────, B⎟
⎝ e_B⋅g₈         a₀         ⎠

## parameter ranges from coexistence system
$C$ is always positive for any values of parameters in the expression

In [9]:
P_0 = Eq(sol[3][1],0)

sol1 = solve(P_0, [g_P,e_P,r_P,m_P,a_P,T_k])
sol1

⎡⎛    -0.120336943441637⋅E_P                -0.120336943441637⋅E_P       -0.12
⎢⎜    ───────────────────────               ───────────────────────      ─────
⎢⎜              T_k            m₀ + r₀                T_k                     
⎢⎜g₀⋅ℯ                       , ───────, r₀⋅ℯ                       , m₀⋅ℯ     
⎣⎝                                g₀                                          

0336943441637⋅E_P       -0.120336943441637⋅E_P      ⎞⎤
──────────────────      ───────────────────────     ⎟⎥
     T_k                          T_k               ⎟⎥
                  , a₀⋅ℯ                       , T_k⎟⎥
                                                    ⎠⎦

and this shows efficiency ($e_P$) of phytoplankton is temperature-independent (as expected/assumed) can only be larger than $\dfrac{m_P + r_P}{g_P}$.  Given that 0 $\leq e_P\leq$ 1, only $g_P\geq (m_P + r_P)$ is accepted.  Given that all phytoplankton is having $g_P>(r_P+m_P)$ in order to have population growth, this implies that all phytoplankton can be used (?), as long as it survives under the given temperature.